In [1]:
from __future__ import print_function

## Cppyy Tutorial

This tutorial introduces the basic concepts for using cppyy, the auomatic Python-C++ generator. To install cppyy on your system, simply run (this may take a while as it will pull in and compile a custom version of LLVM):

```
$ pip install cppyy
```

For further details on the installation, as well as the location of binary wheels, see:
   http://cppyy.readthedocs.io/en/latest/installation.html

To start, import module cppyy. All functionality, including using bound classes, always starts at this top-level.

In [2]:
import cppyy

There are three layers to cppyy: at the top there are the module 'gbl' (the _global_ namespace), a range of helper functions, and a set of sub-modules (such as `py`) that serve specific purposes. Let's start with defining a little helper class in C++ using the helper function `cppdef`, to make the example more interesting:

In [3]:
cppyy.cppdef("""
class Integer1 {
public:
    Integer1(int i) : m_data(i) {}
    int m_data;
};""")

We now have a class 'Integer1'. Note that this class exists on the C++ side and has to follow C++ rules. For example, whereas in Python we can simply redefine a class, we can't do that in C++. Therefore, we will number the `Integer` classes as we go along, to be able to extend the example as we see fit.

Python classes are constructed dynamically. It doesn't matter where or how they are defined, whether in a Python script, "compiled" into a C extension module, or otherwise. Cppyy takes advantage of this fact to generate bindings on-the-fly. This leads to performance advantages for large libraries with thousands of C++ classes; general distribution advantages since, other than the module cppyy itself, no code depends on any specific version of Python; and it enablers, through the Cling backend, interactive access to C++.

To access our first class, find it in gbl, the global namespace:

In [4]:
print(cppyy.gbl.Integer1)

<class cppyy.gbl.Integer1 at 0x1a426c0>


Namespaces have simularities to modules, so we could have imported the class as well.

Bound C++ classes are first-class Python object. We can instantiate them, use normal Python introspection tools, call `help()`, they raise Python exceptions on failure, manage memory through Python's ref-counting and garbage collection, etc., etc. Furthermore, we can use them in conjunction with other C++ classes.

In [5]:
# for convenience, bring Integer1 into __main__
from cppyy.gbl import Integer1

# create a C++ Integer1 object
i = Integer1(42)

# use Python inspection
print("Variable has an 'm_data' data member?", hasattr(i, 'm_data') and 'Yes!' or 'No!')
print("Variable is an instance of int?", isinstance(i, int) and 'Yes!' or 'No!')
print("Variable is an instance of Integer1?", isinstance(i, Integer1) and 'Yes!' or 'No!')

Variable has an 'm_data' data member? Yes!
Variable is an instance of int? No!
Variable is an instance of Integer1? Yes!


In [6]:
# pull in the STL vector class
from cppyy.gbl.std import vector

# create a vector of Integer1 objects; note how [] instantiates the template and () instantiates the class
v = vector[Integer1]()

# populate it
v += [Integer1(j) for j in range(10)]

# display our vector
print(v)

<cppyy.gbl.std.vector<Integer1> object at 0x336ddb0>


Hum, that doesn't look very pretty. However, since Integer1 is now a Python class we can decorate it, with a custom `__repr__` function (we'll punt on the `vector` and instead convert it to a Python `list` for printing).

In [7]:
# add a custom conversion for printing
Integer1.__repr__ = lambda self: repr(self.m_data)

# now try again (note the conversion of the vector to a Python list)
print(list(v))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


As we have seen so far, automatic bindings are simple and easy to use. However, even though they are first-class Python objects, they do have some rough C++ edges left. There is some _pythonization_ going on in the background: the vector, for example, played nice with `+=` and the list conversion. But for presenting your own classes to end-users, specific pythonizations are desirable. To have this work correctly with lazy binding, a callback-based API exists.

Now, it's too late for Integer1, so let's create Integer2, which lives in a namespace and in addition has a conversion feature.

In [8]:
# create an Integer2 class, living in namespace Math
cppyy.cppdef("""
namespace Math {
    class Integer2 : public Integer1 {
    public:
        using Integer1::Integer1;
        operator int() { return m_data; }
    };
}""")

In [9]:
# prepare a pythonizor
def pythonizor(klass, name):
    # A pythonizor receives the freshly prepared bound C++ class, and a name stripped down to
    # the namespace the pythonizor is applied. Also accessible are klass.__name__ (for the
    # Python name) and klass.__cppname__ (for the C++ name)
    if name == 'Integer2':
        klass.__repr__ = lambda self: repr(self.m_data)

# install the pythonizor as a callback on namespace 'Math' (default is the global namespace)
cppyy.py.add_pythonization(pythonizor, 'Math')

In [10]:
# when we next get the Integer2 class, it will have been decorated
Integer2 = cppyy.gbl.Math.Integer2    # first time a new namespace is used, it can not be imported from
v2 = vector[Integer2]()
v2 += [Integer2(j) for j in range(10)]

# now test the effect of the pythonizor:
print(list(v2))

# in addition, Integer2 has a conversion function, which is automatically recognized and pythonized
i2 = Integer2(13)
print("Converted Integer2 variable:", int(i2))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Converted Integer2 variable: 13


In [11]:
# continue the decoration on the C++ side, by adding an operator+ overload
cppyy.cppdef("""
namespace Math {
    Integer2 operator+(const Integer2& left, const Integer1& right) {
        return left.m_data + right.m_data;
    }
}""")

In [12]:
# now use that fresh decoration (it will be located and bound on use:
k = i2 + i
print(k, i2.m_data + i.m_data)

55 55
